In [7]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo 
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

In [2]:
#Importing Iris dataset as Pandas dataframe.
iris_dataset = fetch_ucirepo(id=53) 

iris_X = iris_dataset.data.features 
iris_Y = iris_dataset.data.targets 

class_mapping = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}

iris_Y['class'] = iris_Y['class'].map(class_mapping)

C:\Users\Shiyan Liu\AppData\Local\Temp\ipykernel_2140\1879270822.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iris_Y['class'] = iris_Y['class'].map(class_mapping)


In [48]:
class BinningLayer(torch.nn.Module):
    def __init__(self, num_cutting_points, tau, feature_min, feature_max):
        super().__init__()
        
        #Constant binner in the first softmax layer. Not trained
        self.w=torch.Tensor([i+1 for i in range(num_cutting_points+1)])
        
        #Cut points, randomized between the minimum and maximum values for each feature. Length n.
        cut_points=(feature_min + (feature_max - feature_min) * torch.rand(num_cutting_points)).numpy()
        bias_list=torch.Tensor([0]+[-cut_points[i] for i in range(len(cut_points))])
        for i in range(1, len(bias_list)):
            bias_list[i]+=bias_list[i-1]
        self.b = torch.nn.Parameter(bias_list)

        self.tau=tau #Temperature parameter for the first softmax layer.
        self.softmax=torch.nn.Softmax() #Classifier softmax layer.
        
    #Forward propogation.
    def forward(self, x):
        x=torch.add(torch.multiply(self.w, x), self.b)
        x=self.softmax(torch.div(x, self.tau))
        return x
    
class DNDT(torch.nn.Module):
    def __init__(self, X_frame, Y_frame, num_cutting_points=1, tau=0.01):
        super().__init__()
        self.X=X_frame #Features dataframe in pandas.
        self.Y=Y_frame #Targets dataframe in pandas.
        self.num_cutting_points=num_cutting_points #Number of cutting points for each feature.
        self.tau=tau #Temperature parameter for the first softmax layer.
        
        self.range_matrix=self.create_range_matrix() #Dx2 matrix, where each length 2 matrix is the min and
            #max of each feature in your dataset.
        self.binning_layers=self.create_binning_layers() #D matrix, where the i-entry is a NN model for
            #soft binning the i-th feature.
        
        self.num_features=self.X.shape[1] #D
        self.num_classes=max(self.Y[self.Y.columns[0]])+1 #C
        
        self.linear_layer=torch.nn.Linear((self.num_cutting_points+1)**(self.num_features), self.num_classes) #Linear 
            #classifier layer.

        self.softmax=torch.nn.Softmax() #Classifier softmax layer.
        
    #Creates binning layers list.
    def create_binning_layers(self):
        res=[None]*self.X.shape[1]
        for i in range(len(res)):
            res[i]=BinningLayer(self.num_cutting_points, self.tau, self.range_matrix[i][0], self.range_matrix[i][1])
        return res
    
    #Creates range matrix list.
    def create_range_matrix(self):
        res=[[0,0] for i in range(self.X.shape[1])]
        columns=self.X.columns
        i=0
        for column in columns:
            res[i][0]=min(self.X[column])
            res[i][1]=max(self.X[column])
            i+=1
        return res
            
    #Kronecker products all of the softly-binned layers.
    def kronecker_prod(self, x):
        result=x[0]
        for i in range(1, len(x)):
            result=torch.kron(result, x[i])
        return result
    
    #Forward propogation.
    def forward(self, x):
        x = torch.stack([self.binning_layers[i](x[:, i]) for i in range(self.num_features)], dim=1)
        x=self.kronecker_prod(x)
        x=self.linear_layer(x)
        x=self.softmax(x)
        return x
    

In [56]:
deep_decision_tree=DNDT(iris_X, iris_Y, 1)

X=iris_X
Y=iris_Y

X_tensor = torch.tensor(X.values, dtype=torch.float32)
Y_tensor = torch.tensor(Y.values, dtype=torch.long).squeeze()

X_train, X_val, Y_train, Y_val = train_test_split(X_tensor, Y_tensor, test_size=0.2, random_state=42)

train_dataset = TensorDataset(X_train, Y_train)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

val_dataset = TensorDataset(X_val, Y_val)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=True)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(deep_decision_tree.parameters(), lr=0.001)

num_epochs=3

for epoch in range(num_epochs):
    for batch in train_loader:
        optimizer.zero_grad()
        outputs = deep_decision_tree(batch[0])
        loss = loss_function(outputs, batch[1])
        loss.backward()
        optimizer.step()

C:\Users\Shiyan Liu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


RuntimeError: size mismatch (got input: [3], target: [1])